In [0]:
from google.colab import drive, files
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers,losses,optimizers, backend
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from scipy import ndimage as nd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, BatchNormalization, Activation,DepthwiseConv2D, Input, Dense, GlobalAveragePooling2D, Dropout, Flatten, Reshape, MaxPooling2D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPooling2D

from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import binary_accuracy, mae
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg19 import VGG19
import warnings

In [0]:
backend.set_image_data_format('channels_first')
drive.mount('/content/gdrive/')
base_path = "/content/gdrive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


# Feature Extraction using base VGG19 model and pooling layers

In [0]:
def feature_extractor(base_path, plane, base_model, pooling_model, save_path):
   path = base_path + "/" + plane
   save_path = save_path + "/" + plane
   files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
   for f in files:
     temp = np.load(path + "/" + f)
     temp = np.stack([temp]*3, axis=1)
     print(temp.shape)
     slices = temp.shape[0]

     X_train = Input((slices,512))
     train = BatchNormalization()(X_train)
     train = GlobalMaxPooling1D()(train)
     train = Flatten()(train)

     final = models.Model(inputs=X_train, outputs=train)

     features = base_model.predict(temp)
     print(features.shape)
     output = pooling_model.predict(features)
     predictions = final.predict(output.reshape((1,slices,512)))
     print(output.shape)
     print(predictions.shape)

     np.save(save_path + "/" + f, predictions)





In [0]:
base_model = VGG19(weights='imagenet',include_top=False, input_shape=(3,256,256))

features = Input((512,8,8))
gap = GlobalAveragePooling2D()(features)
pooling_model = models.Model(inputs=features, outputs=gap)

feature_extractor(base_path+"Dataset/TestSet/", "coronal", base_model, pooling_model, base_path+"Features/TestSet/")

In [0]:
def train_mrnet(base_path, plane, label, model):
  total = base_path + "/" + plane
  Y_train = np.load("/content/gdrive/My Drive/Dataset/TrainingSet/" + label + ".npy")[:,1]
  files = [f for f in sorted(os.listdir(total)) if f.endswith(".npy")]
  X_train = None
  mc = ModelCheckpoint('/content/gdrive/My Drive/Models/MRNET_'+plane+'_'+label+'.h5', monitor='val_accuracy', mode='max', verbose=2, save_best_only=True)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)
  lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                         patience=5, min_lr=0.0001)

  for f in files:
    temp = np.load(total + "/" + f)
    if X_train is None:
      X_train = temp
    else:
      X_train = np.vstack((X_train, temp))
  print(X_train.shape)

  X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True)

  weights = class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
  class_weights = dict(enumerate(weights))
  validation_weights = class_weight.compute_sample_weight('balanced', Y_valid)

  history = model.fit(X_train,Y_train, batch_size=1,
                      validation_data=(X_valid, Y_valid, validation_weights),
                      epochs = 100, callbacks = [mc, es, lr], verbose=2, class_weight = class_weights)





# Passing features to dense layers for final classification

In [0]:
inputs = Input((512))
model = Dense(64, activation="relu", kernel_regularizer=l2(0.001))(inputs)
model = Dropout(0.5)(model)
model = Dense(64, activation="relu", kernel_regularizer=l2(0.001))(model)
model = Dropout(0.5)(model)
pred = Dense(1,activation='sigmoid')(model)
mrnet = models.Model(inputs=inputs,outputs=pred)

adam = optimizers.Adam(learning_rate=0.0001)

mrnet.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])  #Should've been F1_score and Recall

train_mrnet(base_path + "Features/TrainingSet", "sagittal", "abnormal", mrnet)

(1130, 512)
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.73451, saving model to /content/gdrive/My Drive/Models/MRNET_sagittal_abnormal.h5
1017/1017 - 3s - loss: 1.7896 - accuracy: 0.5270 - val_loss: 0.7000 - val_accuracy: 0.7345 - lr: 1.0000e-04
Epoch 2/100

Epoch 00002: val_accuracy did not improve from 0.73451
1017/1017 - 3s - loss: 0.8826 - accuracy: 0.5772 - val_loss: 0.7599 - val_accuracy: 0.7257 - lr: 1.0000e-04
Epoch 3/100

Epoch 00003: val_accuracy did not improve from 0.73451
1017/1017 - 3s - loss: 0.8290 - accuracy: 0.6618 - val_loss: 0.7622 - val_accuracy: 0.6372 - lr: 1.0000e-04
Epoch 4/100

Epoch 00004: val_accuracy did not improve from 0.73451
1017/1017 - 3s - loss: 0.8413 - accuracy: 0.6863 - val_loss: 0.7657 - val_accuracy: 0.7257 - lr: 1.0000e-04
Epoch 5/100

Epoch 00005: val_accuracy improved from 0.73451 to 0.75221, saving model to /content/gdrive/My Drive/Models/MRNET_sagittal_abnormal.h5
1017/1017 - 3s - loss: 0.8261 - accuracy: 0.7011 - val_loss

In [0]:
from sklearn.metrics import accuracy_score

def load_test(path):
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  X_test = None
  for f in files:
    temp = np.load(path + "/" + f)
    if X_test is None:
      X_test = temp
    else:
      X_test = np.vstack((X_test, temp))
  
  return X_test



def ensemble_evaluate(modelname,label):
  planes = ["axial", "coronal", "sagittal"]
  plane_predictions = None
  weights = []
  Y_test = np.load("/content/gdrive/My Drive/Dataset/TestSet/" + label + ".npy")[:,1]
  for i,plane in enumerate(planes):
    print("Predicting plane: " + plane)
    # print("Loading Model")
    model = models.load_model("/content/gdrive/My Drive/Models/"+modelname+"_"+plane+"_"+label+".h5")
    # print("Loading Data")
    X_test = load_test("/content/gdrive/My Drive/Features/TestSet/"+plane)
    # print("Evaluating")
    weights.append(model.evaluate(X_test, Y_test, verbose=0)[1])
    predictions = (model.predict(X_test)>0.5).astype(np.int_).ravel()
    # print("Predicting")
    if plane_predictions is None:
      plane_predictions = predictions
    else:
      plane_predictions = np.vstack((plane_predictions, predictions))
    # print(plane_predictions.shape)

  weighted_predictions = np.arange(plane_predictions.shape[1])
  weights = np.array(weights)
  for i in range(plane_predictions.shape[1]):
    weighted_predictions[i] = np.argmax(np.bincount(np.array([plane_predictions[0][i], plane_predictions[1][i], plane_predictions[2][i]]),weights))
  

  ground_truth = np.load("/content/gdrive/My Drive/Dataset/TestSet/"+label+".npy")[:,1]

  print("Accuracy of " + modelname + " model with " + label + " label: ")
  print(accuracy_score(ground_truth, weighted_predictions))

In [0]:
ensemble_evaluate("MRNET","abnormal")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of MRNET model with abnormal label: 
0.8416666666666667


In [0]:
ensemble_evaluate("MRNET","acl")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of MRNET model with acl label: 
0.6083333333333333


In [0]:
ensemble_evaluate("MRNET","meniscus")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of MRNET model with meniscus label: 
0.6916666666666667
